In [2]:
import os
from langchain.llms import OpenAI 
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
huggingface_api_key = os.getenv('HUGGINGFACE_API_KEY')

In [3]:
llm = OpenAI(temperature=0.6, openai_api_key=openai_api_key)

/Users/joeljoseph26/Documents/Education/Machine_and_Deep_Learning/LLMs/OpenAI/OpenAI/env/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [ ]:
text = "What is the capital of India"
print(llm.predict(text))

In [6]:
from langchain import HuggingFaceHub
llm_huggingface = HuggingFaceHub(repo_id="google/flan-t5-large",huggingfacehub_api_token=huggingface_api_key, model_kwargs={"temperature":0,"max_length":64})

/Users/joeljoseph26/Documents/Education/Machine_and_Deep_Learning/LLMs/OpenAI/OpenAI/env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
output = llm_huggingface.predict("Can you tell me the capital of Russia?")
print(output)

moscow


In [8]:
output2 = llm_huggingface.predict("Can you write a poem about AI?")
print(output2)

i love you i love you i love you i love you i love


In [ ]:
#Prompt Templates
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

demo_template = '''Tell me the capital of {country}'''

prompt = PromptTempate(
    input_variables=['country'],
    template = demo_template
)

prompt.format(country='India')
chain = LLMChain(llm=llm,prompt=prompt)
print(chain.run("India"))

In [12]:
capital_prompt = PromptTemplate(
    input_variables=["country"],
    template = "Please enter the capital of {country}."
)

capital_chain = LLMChain(llm=llm,prompt=capital_prompt)


In [14]:

famous_prompt = PromptTemplate(
    input_variables=["capital"],
    template = "Suggest me famous places of {capital}."
)

famous_chain = LLMChain(llm=llm,prompt=famous_prompt)

In [13]:
#SimpleSequentialChain
from langchain.chains import SimpleSequentialChain

parent_chain = SimpleSequentialChain(chains=[capital_chain,famous_chain])
parent_chain.run('India')

In [15]:
from langchain.chains import SequentialChain

capital_prompt = PromptTemplate(
    input_variables=["country"],
    template = "Please enter the capital of {country}."
)

capital_chain = LLMChain(llm=llm,prompt=capital_prompt,verbose=True,output_key=['capital'])

famous_prompt = PromptTemplate(
    input_variables=["capital"],
    template = "Please enter the capital of {capital}."
)

famous_chain = LLMChain(llm=llm,prompt=famous_prompt,verbose=True,output_key=['places'])

parent_seq_chain = SequentialChain(chains=[capital_chain,famous_chain],
                                   input_variables=["country"],
                                   output_variables=["capital","places"]
                                   )

parent_seq_chain({"country": 'India'})

In [19]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage,SystemMessage,AIMessage

chatllm = ChatOpenAI(temperature=0.6, openai_api_key=openai_api_key, model='gpt-3.5-turbo')

In [18]:
chatllm([
    SystemMessage(content="You are a comedian AI assistant"),
    HumanMessage(content="Please  tell me a joke.")
])

/Users/joeljoseph26/Documents/Education/Machine_and_Deep_Learning/LLMs/Langchain-chapter4/Langchain-Model-Deployment-Huggingface-Spaces/Langchain-Model-Deployment-HuggingFace-Spaces/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [20]:
# Prompt Template + LLM + Output Parsers
from langchain.schema import BaseOutputParser
from langchain.prompts.chat import ChatPromptTemplate

In [21]:
class Commanseparatedoutput(BaseOutputParser):
    def parse(self,text:str):
        return text.strip().split(",")

In [23]:
template = '''You are a helpful assistant. When given an input, 
you should generate 5 synonym words in a comma separated list'''

human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",template),
        ("human",human_template)
    ]
)


In [24]:
chain = chat_prompt | chatllm | Commanseparatedoutput()
chain.invoke({'text':"intelligent"})